In [1]:
%pip install transformers
%pip install datasets
%pip install nltk scikit-learn --quiet


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: datasets in c:\python312\lib\site-packages (3.1.0)



Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import libraries
import os
import json
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import (
    BartTokenizer,
    BartForSequenceClassification,
    BartConfig,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset as Dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import EarlyStoppingCallback
# from google.colab import drive
# # Mount Google Drive
# drive.mount('/content/drive')

In [3]:
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.current_device())

1
NVIDIA GeForce RTX 3070
0


In [4]:
# # File Paths
# train_file_path = "/content/drive/MyDrive/nlp_project/train.jsonlist"
# validation_file_path = "/content/drive/MyDrive/nlp_project/validation.jsonlist"
# test_file_path = "/content/drive/MyDrive/nlp_project/test.jsonlist"

# Ellie File Paths
train_file_path = r"C:\Users\aeadu\OneDrive\Documents\NLP Final Project\modelFiles\train.jsonlist"
validation_file_path = r"C:\Users\aeadu\OneDrive\Documents\NLP Final Project\modelFiles\validation.jsonlist"
test_file_path = r"C:\Users\aeadu\OneDrive\Documents\NLP Final Project\modelFiles\test.jsonlist"

# Helper function to process JSON into datasets
def json_to_dataset(path):
    dataset = []
    with open(path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                l = json.loads(line.strip())
                if "verdict" in l and "body" in l:
                    newL = {
                        "labels": 1 if l["verdict"].lower() == "not the asshole" else 0,
                        "text": l["body"]
                    }
                    dataset.append(newL)
            except (json.JSONDecodeError, KeyError) as e:
                print(f"Skipping line due to error: {e}")
                continue
    return Dataset.from_pandas(pd.DataFrame(dataset))

# Load datasets
dataset_train = json_to_dataset(train_file_path)
dataset_validation = json_to_dataset(validation_file_path)
dataset_test = json_to_dataset(test_file_path)

print(dataset_train)
print(dataset_validation)
print(dataset_test)


Dataset({
    features: ['labels', 'text'],
    num_rows: 32000
})
Dataset({
    features: ['labels', 'text'],
    num_rows: 4000
})
Dataset({
    features: ['labels', 'text'],
    num_rows: 4000
})


In [5]:
# Load Tokenizer
print("building tokenizer...")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"], padding="max_length", truncation=True, max_length=1024
    )
    tokenized["labels"] = examples["labels"]
    return tokenized

tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_validation = dataset_validation.map(tokenize_function, batched=True)
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)

print("First Tokenized Example:")
print(tokenized_dataset_train[0])

building tokenizer...


Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

First Tokenized Example:
{'labels': 1, 'text': "[TITLE] [AITA] Hockey Fan [BODY] I went to a hockey game at my college and since I didn't have season tickets to sit in the student section, got the nosebleed seats around center ice about 6 rows from the top.  It was against our rivals, so the seats were packed.  Every time the action came around the boards on our side, I had to stand up to see what was happening.  I also stood when we got close to scoring a goal.  The people behind me were pissed that I kept standing up and told me to just watch what was happening on the jumbotron.  My thoughts were that if I wanted to sit down and watch the game on a screen, I would have stayed at home and watched from my couch.  What are your thoughts; am I the asshole?", 'input_ids': [0, 10975, 47217, 3850, 742, 646, 250, 2068, 250, 742, 8471, 11232, 646, 387, 37588, 742, 38, 439, 7, 10, 5006, 177, 23, 127, 1564, 8, 187, 38, 399, 75, 33, 191, 3308, 7, 2662, 11, 5, 1294, 2810, 6, 300, 5, 8658, 5225, 1

In [6]:
# Metrics computation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    print("Labels:", labels)
    print("Predictions:", predictions)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="binary", zero_division=0
    )
    accuracy = accuracy_score(labels, predictions)

    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

In [7]:

# Model setup with BART
model_config = BartConfig.from_pretrained("facebook/bart-base", num_labels=2)
model = BartForSequenceClassification.from_pretrained("facebook/bart-base", config=model_config)

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Enable CUDA debugging

# Training arguments
training_args = TrainingArguments(
    output_dir= r"C:\Users\aeadu\OneDrive\Documents\NLP Final Project\modelFiles\ModelOutput",
    num_train_epochs=3,  # Train for fewer epochs
    per_device_train_batch_size=16,  # Larger batch size
    per_device_eval_batch_size=128,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,  # Regularization
    warmup_steps=500,  # Gradual learning rate warmup
    lr_scheduler_type="cosine",  # Use linear decay for learning rate
    save_total_limit=1,  # Save only the best model
    load_best_model_at_end=True,  # Load the best model at the end
    logging_dir='./logs',
    logging_steps=50,
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch size
    fp16=True  # Mixed precision training
)


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
print("Training Labels:", [example["labels"] for example in tokenized_dataset_train])
print("Validation Labels:", [example["labels"] for example in tokenized_dataset_validation])

print("Number of labels in model config:", model.config.num_labels)

print("Sample tokenized input:", tokenized_dataset_train[0])


Training Labels: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_validation,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Early stopping
)

# Train the model
trainer.train()

# Evaluate model on the test set
test_results = trainer.evaluate(tokenized_dataset_test)
print(f"Test set results: {test_results}")

# Save the model and tokenizer
model.save_pretrained(r"C:\Users\aeadu\OneDrive\Documents\NLP Final Project\modelFiles\ModelOutput\aita_BART_classifier_model")
tokenizer.save_pretrained(r"C:\Users\aeadu\OneDrive\Documents\NLP Final Project\modelFiles\ModelOutput\aita_BART_classifier_tokenizer")

C:\Users\aeadu\AppData\Local\Temp\ipykernel_25312\2521150245.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/1500 [00:00<?, ?it/s]

In [ ]:
# NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE - NOT IN USE
 # Add sentiment and external features
def add_sentiment_features(dataset):
    sid = SentimentIntensityAnalyzer()

    def compute_sentiment(example):
        sentiment_scores = sid.polarity_scores(example["text"])
        example["sentiment"] = sentiment_scores["compound"]
        return example

    return dataset.map(compute_sentiment)

# Add sentiment features
dataset_train = add_sentiment_features(dataset_train)
dataset_validation = add_sentiment_features(dataset_validation)
dataset_test = add_sentiment_features(dataset_test)

# Normalize features
def normalize_features(dataset):
    scaler = MinMaxScaler()
    sentiment_scores = [example["sentiment"] for example in dataset]
    sentiment_scaled = scaler.fit_transform(np.array(sentiment_scores).reshape(-1, 1)).flatten()
    for i, example in enumerate(dataset):
        example["sentiment"] = sentiment_scaled[i]
    return dataset


dataset_train = normalize_features(dataset_train)
dataset_validation = normalize_features(dataset_validation)
dataset_test = normalize_features(dataset_test)